In [1]:
%load_ext autoreload
%autoreload 2
from gensim.test.utils import datapath
from gensim.models import KeyedVectors
from scipy.spatial.distance import cosine as cosine_distance
from scipy.special import comb as num_combinations
import numpy as np
from itertools import combinations
from tqdm import tqdm
import sys
sys.path.append('../src')
from models import produce_effect_size, calculate_association_metric_for_target_word, get_matrices_from_term_lists

In [2]:
wv_from_bin = KeyedVectors.load_word2vec_format("../data/external/GoogleNews-vectors-negative300.bin", binary=True)  # C bin format

In [10]:
wv_from_bin['science']

array([ 0.0090332 ,  0.12158203,  0.3984375 ,  0.21386719, -0.01672363,
        0.06396484,  0.3125    , -0.23535156, -0.10009766, -0.09326172,
        0.20507812, -0.03222656, -0.21191406,  0.00958252, -0.27734375,
        0.09619141, -0.1875    , -0.04052734,  0.2265625 , -0.23632812,
       -0.04858398,  0.03295898, -0.08056641,  0.12255859,  0.05859375,
       -0.09130859, -0.02685547,  0.02587891, -0.11376953, -0.21972656,
       -0.0378418 , -0.15722656, -0.05981445, -0.01635742,  0.11425781,
       -0.15234375, -0.01086426,  0.29296875,  0.22265625, -0.00245667,
       -0.16308594, -0.11669922, -0.20800781,  0.31835938, -0.08740234,
        0.13378906,  0.15429688,  0.11816406, -0.07470703,  0.09326172,
       -0.32421875,  0.01239014, -0.07226562, -0.10986328, -0.07177734,
        0.16113281,  0.05541992, -0.3984375 , -0.04980469, -0.25195312,
       -0.05249023, -0.05786133,  0.16699219, -0.26953125, -0.04394531,
       -0.14257812,  0.03662109,  0.34765625, -0.10888672, -0.12

In [21]:
# Debug WEAT 6
X_terms = ['John', 'Paul','Mike','Kevin']
Y_terms = ['Amy','Joan','Lisa','Sarah']
A_terms = ['executive','management','professional','corporation']
B_terms = ['home','parents','children','family']

In [29]:
# WEAT 6
X_terms = ['John', 'Paul','Mike','Kevin','Steve','Greg','Jeff','Bill']
Y_terms = ['Amy','Joan','Lisa','Sarah','Diana','Kate','Ann','Donna']
A_terms = ['executive','management','professional','corporation',
               'salary','office','business','career']
B_terms = ['home','parents','children','family',
               'cousins','marriage','wedding','relatives']

In [24]:
# WEAT 7 using Google News word2vec model
X_terms = ['math','algebra','geometry','calculus',
             'equations','computation','numbers','addition']
Y_terms = ['poetry','art','dance','literature',
             'novel','symphony','drama','sculpture']
A_terms = ['male','man','boy','brother',
              'he','him','his','son']
B_terms = ['female','woman','girl','sister',
               'she','her','hers','daughter']

In [26]:
# Weat 8
A_terms = ['brother','father','uncle','grandfather',
          'son','he','his','him']
B_terms = ['sister','mother','aunt','grandmother',
          'daughter','she','hers','her']
X_terms = ['science','technology','physics','chemistry',
          'Einstein','NASA','experiment','astronomy']
Y_terms = ['poetry','art','Shakespeare','dance',
          'literature','novel','symphony','drama']

In [10]:
wv_test_object = {'x_word1':[.1,.2,.3],
            'x_word2':[.4,.5,.6],
            'y_word1':[.2,.4,.6],
            'y_word2':[.1,.3,.5],
            'a_word1':[.5,.7,.9],
            'a_word2':[.4,.6,.8],
            'b_word1':[.3,.6,.9],
            'b_word2':[.4,.7,.9]}

In [12]:
X_terms = ['x_word1', 'x_word2']
Y_terms = ['y_word1', 'y_word2']
A_terms = ['a_word1', 'a_word2']
B_terms = ['b_word1', 'b_word2']
[X_mtx, Y_mtx, A_mtx, B_mtx] = get_matrices_from_term_lists(wv_test_object, X_terms, Y_terms, A_terms, B_terms)

In [61]:
produce_effect_size(wv_test_object, X_terms, Y_terms, A_terms, B_terms)

1.1899801283181513

# Test Statistic

In [66]:
x1_vec = wv_test_object['x_word1']
s1 = calculate_association_metric_for_target_word(x1_vec, A_mtx, B_mtx)

In [68]:
x2_vec = wv_test_object['x_word2']
s2 = calculate_association_metric_for_target_word(x2_vec, A_mtx, B_mtx)

In [74]:
y1_vec = wv_test_object['y_word1']
sy1 = calculate_association_metric_for_target_word(y1_vec, A_mtx, B_mtx)
y2_vec = wv_test_object['y_word2']
sy2 = calculate_association_metric_for_target_word(y2_vec, A_mtx, B_mtx)

In [75]:
print(s1, s2, s1+s2 - (sy1+sy2))

-0.007639423266564194 0.013706608780193585 0.03175706038191817


In [13]:
def produce_test_statistic(wv_obj, X_terms, Y_terms, A_terms, B_terms):
    [X_mtx, Y_mtx, A_mtx, B_mtx] = get_matrices_from_term_lists(wv_obj, X_terms, Y_terms, A_terms, B_terms)
    x_associations = np.apply_along_axis(lambda x_vec: calculate_association_metric_for_target_word(x_vec, A_mtx, B_mtx), 1, X_mtx)
    y_associations = np.apply_along_axis(lambda y_vec: calculate_association_metric_for_target_word(y_vec, A_mtx, B_mtx), 1, Y_mtx)
    return np.sum(x_associations) - np.sum(y_associations)
produce_test_statistic(wv_test_object, X_terms, Y_terms, A_terms, B_terms)

0.03175706038191817

In [6]:
def get_complements(x_union_y):
    '''Generator function that yields pairs of equal-size disjoint subsets
    of x_union_y.
    x_union_y should a set type.'''
    already_seen = set()
    for seq in combinations(x_union_y, len(x_union_y)//2):
        complement = frozenset(x_union_y.difference(seq))
        #already_seen.append(complement)
        already_seen.add(complement)
        if frozenset(seq) in already_seen:
            continue
        yield (seq, complement)

In [5]:
x_union_y = {'a','b','c','d'}
for tup in get_complements(x_union_y):
    print(tup)

(('c', 'd'), frozenset({'b', 'a'}))
(('c', 'b'), frozenset({'d', 'a'}))
(('c', 'a'), frozenset({'d', 'b'}))


In [30]:
def get_p_value(wv_obj, X_terms, Y_terms, A_terms, B_terms):
    x_union_y = set(X_terms).union(set(Y_terms))
    total_terms = len(x_union_y)
    total_pairs = 0
    high_test_statistics = 0
    comparison_statistic = produce_test_statistic(wv_obj, X_terms, Y_terms, A_terms, B_terms)
    print(comparison_statistic)
    for (X_i_terms, Y_i_terms) in tqdm(get_complements(x_union_y), total=num_combinations(total_terms, total_terms/2)/2):
        total_pairs += 1
        test_statistic = produce_test_statistic(wv_obj, X_i_terms, Y_i_terms, A_terms, B_terms)
        print(test_statistic)
        if (test_statistic > comparison_statistic): high_test_statistics += 1
    print(high_test_statistics)
    print(total_pairs)
    return (float(high_test_statistics) / float(total_pairs))
get_p_value(wv_from_bin, X_terms, Y_terms, A_terms, B_terms)

1.2516100144284792


  0%|                                                                                       | 0/6435.0 [00:00<?, ?it/s]

-0.3251369113140754
-0.2856544395481251


  0%|                                                                               | 2/6435.0 [00:00<05:33, 19.26it/s]

-0.6654130266551874
-0.3365131830369137


...

100%|███████████████████████████████████████████████████████████████████████████▉| 6433/6435.0 [05:41<00:00, 18.10it/s]

0.3442298857999049
-0.03552870130715746


100%|████████████████████████████████████████████████████████████████████████████| 6435/6435.0 [05:41<00:00, 18.82it/s]


0
6435


0.0

# Results

Weat 6: 0 (0/6435)
Weat 7: 0.02905 (187/6435)
Weat 8: 0.0003108 (2/6435)